In [2]:
import geopandas
import r5py
import shapely
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime

import os
import snman
from snman import osmnx_customized as oxc
from snman.constants import *

PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)

CRS_internal = 2056      # for Zurich

C:\Users\lballo\AppData\Roaming\Python\Python39\site-packages\osmnx\utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


In [3]:
def convert_gdf_to_parquet(gdf, parquet):
    gdf = snman.io.import_geofile_to_gdf(gdf)
    gdf.to_parquet(parquet, compression='gzip')
    return gdf

In [4]:
# Convert street graph to parquet
convert_gdf_to_parquet(
    os.path.join(outputs_path, 'G_edges.gpkg'),
    os.path.join(outputs_path, 'G_edges.gzip')
)

convert_gdf_to_parquet(
    os.path.join(outputs_path, 'G_nodes.gpkg'),
    os.path.join(outputs_path, 'G_nodes.gzip')
)

In [2]:
# Convert statpop to parquet
statpop = convert_gdf_to_parquet(
    os.path.join(inputs_path, 'switzerland', 'switzerland', 'statpop', 'statpop2017_reduced_columns.gpkg'),
    os.path.join(inputs_path, 'switzerland', 'switzerland', 'statpop', 'statpop2017_reduced_columns.gzip')
)

statpop.head()

In [3]:
# Convert statent to parquet

statent = convert_gdf_to_parquet(
    os.path.join(inputs_path, 'switzerland', 'switzerland', 'statent', 'STATENT_2021_ebc_10_reduced_fields.gpkg'),
    os.path.join(inputs_path, 'switzerland', 'switzerland', 'statent', 'STATENT_2021_ebc_10_reduced_fields.gzip')
)

statent.head()

In [10]:
# Join statpop with statent
statpop = gpd.read_parquet(
    os.path.join(inputs_path, 'switzerland', 'switzerland', 'statpop', 'statpop2017_reduced_columns.gzip')
)

statpop = gpd.sjoin_nearest(statpop, statent[['RELI', 'geometry']], how='left', max_distance=100, rsuffix='right', distance_col='distance_to_statent_point')
statpop.rename(columns={'RELI': 'statent_id'}, inplace=True)
statpop.set_index('record', inplace=True)

statpop.to_parquet(
    os.path.join(inputs_path, 'switzerland', 'switzerland', 'statpop', 'statpop2017_with_statent_reduced_columns.gzip'),
    compression='gzip'
)

statpop.head()

,statyear,yearofbirth,sex,maritalstatus,residencepermit,age,residentpermit,federalbuildingid,geometry,index_right,statent_id,distance_to_statent_point
record,,,,,,,,,,,,
12,2017,1962,1,2,302,55,3,896883164,POINT (2674380.001 1250094.001),18914.0,67442501.0,20.879700
15,2017,1966,2,1,70102,51,7,899882330,POINT (2672299.001 1251208.001),17671.0,67232512.0,8.063317
16,2017,1984,1,1,13,33,13,899882659,POINT (2673496.001 1250355.001),NaN,NaN,NaN
20,2017,1992,1,2,201,25,2,899882087,POINT (2671984.001 1250827.001),17414.0,67202508.0,31.385383
21,2017,1994,2,2,201,23,2,899882087,POINT (2671984.001 1250827.001),17414.0,67202508.0,31.385383
